
# Tarefa - capturar e armazenar as informações de temperatura



* Depois que o dispositivo IoT estiver publicando telemetria, o código do servidor pode ser escrito para assinar esses dados e armazená-los. 
* Em vez de salvá-los em um banco de dados, o código do servidor os salvará em um arquivo Comma Separated Values ​​(CSV). 
    * Os arquivos CSV armazenam dados como linhas de valores como texto, com cada valor separado por uma vírgula e cada registro em uma nova linha. 
    * CSV é uma maneira conveniente, legível por humanos e bem suportada de salvar dados como um arquivo.

O arquivo CSV terá duas colunas - *data* e *temperatura*. A coluna *data* é definida como a data e hora atuais em que a mensagem foi recebida pelo servidor, a *temperatura* vem da mensagem de telemetria.

1. Repita os passos da lição sobre como criar um servidor para criar o código do servidor para assinar a telemetria. Você não precisa adicionar código para publicar comandos.

    Os passos para isso são:

    * Configurar e ativar um ambiente virtual Python

    * Instale o pacote pip paho-mqtt

    * Escreva o código para escutar mensagens MQTT publicadas no tópico telemetria

      > ⚠️ Você pode consultar [as instruções na lição 4 para criar um aplicativo Python para receber telemetria, se necessário](https://github.com/microsoft/IoT-For-Beginners/blob/main/1-getting-started/lessons/4-connect-internet/README.md#receive-telemetry-from-the-mqtt-broker).

    Nomeie a pasta para este projeto `temperature-sensor-server`.

1. Certifique-se de que o `client_name` reflete este projeto:

   ```cpp
    client_name = id + 'temperature_sensor_server'
    ```

1. Adicione as seguintes importações ao topo do arquivo, abaixo das importações existentes:

    ```python
    from os import path
    import csv
    from datetime import datetime
    ```

    Isso importa uma biblioteca para leitura de arquivos, uma biblioteca para interagir com arquivos CSV e uma biblioteca para ajudar com datas e horas.

1. Adicione o seguinte código antes da função `handle_telemetry`:

    ```python
    temperature_file_name = 'temperature.csv'
    fieldnames = ['date', 'temperature']

    if not path.exists(temperature_file_name):
        with open(temperature_file_name, mode='w') as csv_file:
            writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
            writer.writeheader()
    ```

    Este código declara algumas constantes para o nome do arquivo para escrever, e o nome dos cabeçalhos de coluna para o arquivo CSV. A primeira linha de um arquivo CSV tradicionalmente contém cabeçalhos de coluna separados por vírgulas.

    O código então verifica se o arquivo CSV já existe. Se não existir, ele é criado com os cabeçalhos de coluna na primeira linha.

1. Adicione o seguinte código ao final da função `handle_telemetry`:

    ```python
    with open(temperature_file_name, mode='a') as temperature_file:        
        temperature_writer = csv.DictWriter(temperature_file, fieldnames=fieldnames)
        temperature_writer.writerow({'date' : datetime.now().astimezone().replace(microsecond=0).isoformat(), 'temperature' : payload['temperature']})
    ```

    Este código abre o arquivo CSV e, em seguida, acrescenta uma nova linha no final. A linha tem os dados e a hora atuais formatados em um formato legível por humanos, seguidos pela temperatura recebida do dispositivo IoT. Os dados são armazenados no [formato ISO 8601](https://wikipedia.org/wiki/ISO_8601) com o fuso horário, mas sem microssegundos.

1. Execute este código da mesma forma que antes, certificando-se de que seu dispositivo IoT esteja enviando dados. Um arquivo CSV chamado `temperature.csv` será criado na mesma pasta. Se você visualizá-lo, verá data/hora e medições de temperatura:

    ```output
    date,temperature
    2021-04-19T17:21:36-07:00,25
    2021-04-19T17:31:36-07:00,24
    2021-04-19T17:41:36-07:00,25
    ```

1. Execute este código por um tempo para capturar dados. O ideal é que você execute isso por um dia inteiro para reunir dados suficientes para cálculos de GDD.

    > 💁 Se você estiver usando o Dispositivo IoT Virtual, marque a caixa de seleção aleatória e defina um intervalo para evitar obter a mesma temperatura sempre que o valor da temperatura for retornado.
    ![Selecione a caixa de seleção aleatória e defina um intervalo](https://github.com/microsoft/IoT-For-Beginners/blob/main/images/select-the-random-checkbox-and-set-a-range.png?raw=true)

    > 💁 Se você quiser executar isso por um dia inteiro, precisará garantir que o computador no qual o código do servidor está sendo executado não entre em modo de espera, alterando suas configurações de energia ou executando algo como [este script Python para manter o sistema ativo](https://github.com/jaqsparow/keep-system-active).
    
> 💁 Você pode encontrar este código na pasta [code-server/temperature-sensor-server](https://github.com/microsoft/IoT-For-Beginners/blob/main/2-farm/lessons/1-predict-plant-growth/code-server/temperature-sensor-server).


In [ ]:
import time
import random
import json
import paho.mqtt.client as mqtt
from os import path
import csv
from datetime import datetime


# Configuração do broker MQTT
broker = 'mqtt.eclipseprojects.io'  # Endereço do broker MQTT
port = 1883  # Porta padrão do MQTT
telemetry_topic = "capacitacao-iot/telemetria"  # Tópico de telemetria para envio e recebimento de dados

# Gera um identificador único para o cliente MQTT
client_id = f'capacitacao-iot-mqtt-nightlight_client-{random.randint(0, 100000)}'
print("Client ID: " + client_id)

# Função de callback chamada quando o cliente se conecta ao broker MQTT
def on_connect(client, userdata, flags, reason_code, properties):
    # Verifica se a conexão foi bem-sucedida ou se houve falha
    if reason_code != 0:
        print(f"Falha ao conectar: {reason_code}. 'loop_forever()' tentará reconectar.")
    else:
        print("Conectado ao broker MQTT!")
        # sobrescreve no tópico de telemetria
        client.subscribe(telemetry_topic)
        
# Abre um arquivo csv onde serão armazenadas as leituras de temperatura recebedias.
temperature_file_name = 'temperature.csv'
fieldnames = ['date', 'temperature', 'sensor_id']

if not path.exists(temperature_file_name):
    with open(temperature_file_name, mode='w') as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader()

# Função de callback chamada quando uma mensagem é recebida no tópico inscrito
def handle_telemetry(client, userdata, message):
    # Decodifica e exibe o payload da mensagem recebida
    payload = json.loads(message.payload.decode())
    print("Mensagem recebida:", payload)

    with open(temperature_file_name, mode='a') as temperature_file:        
        temperature_writer = csv.DictWriter(temperature_file, fieldnames=fieldnames)
        temperature_writer.writerow({'date' : datetime.now().astimezone().replace(microsecond=0).isoformat(), 
                                     'temperature' : payload['temperature'],
                                     'sensor_id': payload['sensor_id']})

# Cria o cliente MQTT e configura os callbacks
mqttc = mqtt.Client(mqtt.CallbackAPIVersion.VERSION2,client_id=client_id)
mqttc.on_connect = on_connect  # Associa a função de callback ao evento de conexão
mqttc.on_message = handle_telemetry  # Associa a função de callback para mensagens recebidas

# Configura dados adicionais de usuário, se necessário
mqttc.user_data_set([])

# Conecta ao broker MQTT
mqttc.connect(broker, port)

# Inicia o loop do cliente MQTT (bloqueante), que gerencia eventos de conexão e mensagens
mqttc.loop_forever()

Client ID: capacitacao-iot-mqtt-nightlight_client-29613
Conectado ao broker MQTT!
Mensagem recebida: {'temperature': 39.84, 'sensor_id': 'capacitacao-iot-temperature-sensor-client8980'}
Mensagem recebida: {'temperature': 35.38, 'sensor_id': 'capacitacao-iot-temperature-sensor-client8980'}
Mensagem recebida: {'temperature': 48.3, 'sensor_id': 'capacitacao-iot-temperature-sensor-client8980'}
Mensagem recebida: {'temperature': 41.44, 'sensor_id': 'capacitacao-iot-temperature-sensor-client8980'}
Mensagem recebida: {'temperature': 89.17, 'sensor_id': 'capacitacao-iot-temperature-sensor-client8980'}
Mensagem recebida: {'temperature': 6.56, 'sensor_id': 'capacitacao-iot-temperature-sensor-client8980'}
Mensagem recebida: {'temperature': 38.41, 'sensor_id': 'capacitacao-iot-temperature-sensor-client8980'}
Mensagem recebida: {'temperature': 49.58, 'sensor_id': 'capacitacao-iot-temperature-sensor-client8980'}
Mensagem recebida: {'temperature': 21.18, 'sensor_id': 'capacitacao-iot-temperature-sens


### Tarefa - calcular GDD usando os dados armazenados

Depois que o servidor captura os dados de temperatura, o GDD de uma planta pode ser calculado.

Os passos para fazer isso manualmente são:

1. Encontre a temperatura base para a planta. Por exemplo, para morangos a temperatura base é 10°C.

1. No `temperature.csv`, encontre as temperaturas mais altas e mais baixas do dia

1. Use o cálculo GDD fornecido anteriormente para calcular GDD

Por exemplo, se a temperatura mais alta do dia for 25°C e a mais baixa for 12°C:

![GDD = 25 + 12 dividido por 2, então subtraia 10 do resultado dando 8,5](https://github.com/microsoft/IoT-For-Beginners/blob/main/images/gdd-calculation-strawberries.png?raw=true)

* 25 + 12 = 37
* 37 / 2 = 18,5
* 18,5 - 10 = 8,5

Portanto, os morangos receberam **8,5** GDD. Os morangos precisam de cerca de 250 GDD para dar frutos, então ainda falta um tempo.

---

## 🚀 Desafio

As plantas precisam de mais do que calor para crescer. Que outras coisas são necessárias?

Para estes, descubra se há sensores que podem medí-los. E quanto aos atuadores para controlar esses níveis? Como você juntaria um ou mais dispositivos IoT para otimizar o crescimento das plantas?


## Revisão e autoestudo

* Leia mais sobre agricultura digital na [página da Wikipedia sobre agricultura digital](https://wikipedia.org/wiki/Digital_agriculture). Leia também mais sobre agricultura de precisão na [página da Wikipedia sobre agricultura de precisão](https://wikipedia.org/wiki/Precision_agriculture).
* O cálculo completo de graus-dia de crescimento é mais complicado do que o simplificado fornecido aqui. Leia mais sobre a equação mais complicada e como lidar com temperaturas abaixo da linha de base na [página da Wikipedia sobre graus-dia de crescimento](https://wikipedia.org/wiki/Growing_degree-day).
* A comida pode ser escassa no futuro, mesmo que ainda usemos os mesmos métodos de agricultura. Aprenda mais sobre técnicas de agricultura de alta tecnologia neste [vídeo Hi-Tech Farms of Future no YouTube](https://www.youtube.com/watch?v=KIEOuKD9KX8).
